## Load modules and data

In [1]:
#Base
import sys, os
import numpy as np
import xarray as xr
import pandas as pd
import scipy.stats as st
from tqdm.notebook import trange, tqdm

#Plot
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import matplotlib.gridspec as gridspec

#V1DD
from allen_v1dd.client import OPhysClient, OPhysSession
from allen_v1dd.stimulus_analysis import StimulusAnalysis
from allen_v1dd.stimulus_analysis.correlation import calculate_correlation, ProgressBar
from allen_v1dd.stimulus_analysis.natural_scenes import NaturalScenes
from allen_v1dd.stimulus_analysis.natural_movie import NaturalMovie
from allen_v1dd.stimulus_analysis.drifting_gratings import DriftingGratings

%matplotlib inline
%load_ext autoreload
%autoreload 2

PlotDir = '/home/david.wyrick/projects/V1DD/plots'

In [2]:
## Load data
base_folder = r"\\allen\programs\mindscope\workgroups\surround\v1dd_in_vivo_new_segmentation\data" #Server
base_folder = "/data/v1dd_in_vivo_new_segmentation" # Local on robinson for golden mouse
client = OPhysClient(base_folder)

In [3]:
##===== Load session from golden mouse =====##
mouse_id = 409828; col = 1; vol = 3; plane = 3

sess = client.load_ophys_session(mouse=mouse_id, column=col, volume=vol)
print(f"Loaded ophys session {sess.get_session_id()} (mouse {sess.get_mouse_id()}, column {sess.get_column_id()}, volume {sess.get_volume_id()})")


Loaded ophys session M409828_13 (mouse 409828, column 1, volume 3)


In [4]:
rois = np.array(sess.get_rois(plane))
mask = sess.is_roi_valid(plane)

In [5]:
stim_table, stim_meta = sess.get_stimulus_table("natural_movie")
stim_table

,start,end,frame
0,1343.707764,1343.724487,0
1,1343.741211,1343.757935,1
2,1343.774536,1343.791260,2
3,1343.807983,1343.824707,3
4,1343.841309,1343.857910,4
...,...,...,...
29695,3537.367676,3537.384277,895
29696,3537.401123,3537.417725,896
29697,3537.434326,3537.451172,897
29698,3537.467773,3537.484375,898


In [6]:
## Create stimulus object for natural scenes analysis
nm_events = NaturalMovie(sess, 3, trace_type="events")

#Calculate metrics 
nm_events_metrics = nm_events.metrics

  0%|          | 0/1482 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Run on all sessions

In [4]:
mouse = 409828 # selected for EM
column = 1

session_ids = client.get_all_session_ids()
# session_ids = [s for s in session_ids if s.startswith(f"M{mouse}") and s[8] == str(column) and s[9] == "3"]
session_ids = [s for s in session_ids if s.startswith(f"M{mouse}") and s[8] == str(column)]

print("Sessions to load:")
print(session_ids)

Sessions to load:
['M409828_11', 'M409828_12', 'M409828_13', 'M409828_14', 'M409828_15', 'M409828_16', 'M409828_17', 'M409828_18', 'M409828_19', 'M409828_1a', 'M409828_1b', 'M409828_1c', 'M409828_1d', 'M409828_1e', 'M409828_1f']


In [5]:
sessions = []

for session_id in tqdm(session_ids):
    session = client.load_ophys_session(session_id=session_id)
    sessions.append(session)

print(f"Loaded {len(sessions)} ophys sessions")

  0%|          | 0/15 [00:00<?, ?it/s]

Loaded 15 ophys sessions


In [10]:
plane_info = [] # holds (mouse, column, volume, plane)
nm = []

with tqdm(desc="Loading natural movie analyses", total=sum(len(s.get_planes()) for s in sessions)) as pbar:
    for session in sessions:
        for plane in session.get_planes():
            plane_info.append((session.get_mouse_id(), session.get_column_id(), session.get_volume_id(), plane))
            nm.append(NaturalMovie(session, plane))

            pbar.update()

Loading natural scene analyses:   0%|          | 0/40 [00:00<?, ?it/s]

In [12]:
RECOMPUTE_METRICS = True
DF_SAVE_DIR = r"/home/david.wyrick/Git/allen_v1dd/dataframes"

if RECOMPUTE_METRICS:

    # Concatenate across the different planes
    print("Concatenating natural movie metrics...")
    nm_metrics = StimulusAnalysis.concat_metrics(nm)
 
    print("Done.", len(nm_metrics), "total ROIs.")
    nm_metrics.to_csv(os.path.join(DF_SAVE_DIR, "nm_metrics_col1.csv"))
nm_metrics

Concatenating ns12 metrics...
